# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [89]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
import sys
sys.path.insert(0, "/Users/Iceda/OneDrive/Desktop/ClassHomework/keys")
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [90]:
weather_df = pd.read_csv("../output_data/weather_data.csv")

weather_df.head()

,City_ID,City,Country,Datetime,Lat,Lng,Temperature,Humidity,Cloud Coverage,Wind Speed
0,0,taolanaro,IN,2020-04-15 04:32:38,21.12,70.12,91.38,35.0,18.0,11.97
1,1,mangrol,GB,2020-04-15 04:32:38,54.33,-2.75,41.00,80.0,90.0,1.99
2,2,kendal,MV,2020-04-15 04:32:38,-0.60,73.08,84.97,65.0,34.0,1.86
3,3,hithadhoo,CN,2020-04-15 04:32:38,45.55,131.88,54.39,34.0,0.0,10.07
4,4,tabiauea,GL,2020-04-15 04:32:38,65.61,-37.64,24.80,68.0,2.0,1.12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [91]:
# Store latitude and longitude in locations
places = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df.Humidity.astype('float')

In [92]:
# Plot Heatmap
fig = gmaps.figure(center = (0,0), zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(places, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [93]:
ideal_df = weather_df.loc[(weather_df.Temperature > 70) & (weather_df.Temperature <= 80) &
                         (weather_df['Cloud Coverage'] == 0) & (weather_df['Wind Speed'] < 10)]

ideal_df = ideal_df.dropna(0)

ideal_df

,City_ID,City,Country,Datetime,Lat,Lng,Temperature,Humidity,Cloud Coverage,Wind Speed
139,139,beisfjord,CN,2020-04-15 04:33:02,27.73,112.53,79.03,48.0,0.0,8.37
143,143,constitucion,CN,2020-04-15 04:33:02,30.30,108.85,71.58,30.0,0.0,2.93
217,217,deputatskiy,NG,2020-04-15 04:33:15,12.87,11.05,77.00,15.0,0.0,9.13
358,358,ibra,SA,2020-04-15 04:33:38,25.02,37.27,74.79,45.0,0.0,7.76
540,540,mersing,CN,2020-04-15 04:34:08,27.81,114.67,78.44,45.0,0.0,4.32


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [94]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ''

In [95]:
params = {"radius": 5000, 
          "type": "lodging",
          "key": g_key}

for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)
    
    #print the url, avoid doing for public github repos in order to avoid exposing key
#     print(hotel_name.url)
    
    hotel_name = hotel_name.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        print("Missing value... skipping...")

In [96]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [97]:
# Add marker layer ontop of heat map
symbols = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(symbols)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))